# 读取数据集

In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('./data/train_set.csv')
test = pd.read_csv('./data/test_set.csv')

In [3]:
train.head()

,id,article,word_seg,class
0,0,7368 1252069 365865 755561 1044285 129532 1053...,816903 597526 520477 1179558 1033823 758724 63...,14
1,1,581131 165432 7368 957317 1197553 570900 33659...,90540 816903 441039 816903 569138 816903 10343...,3
2,2,7368 87936 40494 490286 856005 641588 145611 1...,816903 1012629 957974 1033823 328210 947200 65...,12
3,3,299237 760651 299237 887082 159592 556634 7489...,563568 1239563 680125 780219 782805 1033823 19...,13
4,4,7368 7368 7368 865510 7368 396966 995243 37685...,816903 816903 816903 139132 816903 312320 1103...,12


# TFIDF构建文本特征

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
word_vec = TfidfVectorizer(analyzer='word',
            ngram_range=(1,2),
            min_df=3, 
            max_df=0.9,
            use_idf=True,
            smooth_idf=True, 
            sublinear_tf=True)

In [6]:
train_term_doc = word_vec.fit_transform(train['word_seg'])

In [7]:
test_term_doc = word_vec.transform(test['word_seg'])

In [29]:
train_term_doc

<102277x2820641 sparse matrix of type '<class 'numpy.float64'>'
	with 75739932 stored elements in Compressed Sparse Row format>

In [30]:
train['label'].head()

0    13
1     2
2    11
3    12
4    11
Name: label, dtype: int64

# 逻辑LightGBM模型

In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
lb = LabelEncoder()
train['label'] = lb.fit_transform(train['class'].tolist())[0,...18]

In [16]:
from sklearn.model_selection import train_test_split
#cross_validation
#切分训练集 验证集
X_train, X_eval, y_train, y_eval  = train_test_split(train_term_doc,train['label'],test_size=0.2,shuffle=True,random_state=2019)

In [ ]:
# X_train, X_eval, = train_test_split(train_term_doc,test_size=0.2,shuffle=True,random_state=2019)
# y_train, y_eval  = train_test_split(train['label'],test_size=0.2,shuffle=True,random_state=2019)
#错误的

In [32]:
X_train,X_eval

(<81821x2820641 sparse matrix of type '<class 'numpy.float64'>'
 	with 60544982 stored elements in Compressed Sparse Row format>,
 <20456x2820641 sparse matrix of type '<class 'numpy.float64'>'
 	with 15194950 stored elements in Compressed Sparse Row format>)

In [33]:
y_train.shape,y_eval.shape

((81821,), (20456,))

In [13]:
import lightgbm as lgb
#sklearn api

In [19]:
model =lgb.LGBMClassifier(boosting_type='gbdt', 
                   num_leaves=2**5,
                   max_depth=-1, 
                   learning_rate= 0.1,
                   n_estimators=2000,
                   objective='multiclass',
                   subsample=0.7,#
                   colsample_bytree=0.5,#
                   reg_lambda=10,#l2
                   n_jobs=16, #
                   num_class=19,#
                   silent=True, 
                   random_state=2019,
#                    class_weight=20,
                   colsample_bylevel=0.5,
                   min_child_weight=1.5,
                   metric='multi_logloss'
                  )
model.fit(X_train,y_train,eval_set=(X_eval,y_eval), early_stopping_rounds=100)

[1]	valid_0's multi_logloss: 2.59854
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's multi_logloss: 2.40874
[3]	valid_0's multi_logloss: 2.262
[4]	valid_0's multi_logloss: 2.14355
[5]	valid_0's multi_logloss: 2.04217
[6]	valid_0's multi_logloss: 1.95521
[7]	valid_0's multi_logloss: 1.87851
[8]	valid_0's multi_logloss: 1.80973
[9]	valid_0's multi_logloss: 1.74865
[10]	valid_0's multi_logloss: 1.69333
[11]	valid_0's multi_logloss: 1.64411
[12]	valid_0's multi_logloss: 1.60052
[13]	valid_0's multi_logloss: 1.55825
[14]	valid_0's multi_logloss: 1.51969
[15]	valid_0's multi_logloss: 1.48463
[16]	valid_0's multi_logloss: 1.45241
[17]	valid_0's multi_logloss: 1.42353
[18]	valid_0's multi_logloss: 1.396
[19]	valid_0's multi_logloss: 1.37042
[20]	valid_0's multi_logloss: 1.34695
[21]	valid_0's multi_logloss: 1.32496
[22]	valid_0's multi_logloss: 1.30468
[23]	valid_0's multi_logloss: 1.28517
[24]	valid_0's multi_logloss: 1.26685
[25]	valid_0's multi_logloss: 1.24974


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bylevel=0.5,
        colsample_bytree=0.5, importance_type='split', learning_rate=0.1,
        max_depth=-1, metric='multi_logloss', min_child_samples=20,
        min_child_weight=1.5, min_split_gain=0.0, n_estimators=2000,
        n_jobs=16, num_class=19, num_leaves=32, objective='multiclass',
        random_state=2019, reg_alpha=0.0, reg_lambda=10, silent=True,
        subsample=0.7, subsample_for_bin=200000, subsample_freq=0)

# 线下验证结果

In [34]:
from sklearn.metrics import f1_score
#[1,2,3,2,1,3]
#[1,2,3,1,1,3]
def cal_macro_f1(y_true,y_pred):
    score = f1_score(y_true,y_pred,average='macro')
    return score

In [35]:
eval_prob = model.predict_proba(X_eval) #[10,0227*19]
eval_pred = np.argmax(eval_prob,axis=1)#[]
eval_pred = lb.inverse_transform(eval_pred)[]
score = cal_macro_f1(lb.inverse_transform(y_eval),eval_pred)
print("validation score is",score)
#0.7539065473958163

validation score is 0.7539065473958163


In [36]:
test_prob = model.predict_proba(test_term_doc)

# 结果提交

In [37]:
test_pred = np.argmax(test_prob,axis=1)
test['class'] = lb.inverse_transform(test_pred)

In [38]:
test[["id","class"]].to_csv("submission_baseline_lgb.csv",index=False,header=True,encoding='utf-8')